In [ ]:
import numpy as np
import graphinglib as gl
import scipy as sp

from typing import Self

In [ ]:
class Micrograph:
    def __init__(self, data: np.ndarray, index: int) -> Self:
        self.data = data
        self._index = index
        self.fft = np.fft.fftshift(np.fft.fft2(data))

    @classmethod
    def load(cls, filename: str) -> Self:
        return cls(gl.Heatmap(filename).image, int(filename[-5]))
    
    @property
    def data_plot(self) -> gl.Heatmap:
        return gl.Heatmap(self.data, aspect_ratio="equal", color_map="gray", show_color_bar=False)

    @property
    def fft_plot(self) -> gl.Heatmap:
        return gl.Heatmap(np.abs(self.fft), aspect_ratio="equal", vmax=1e5, color_map="gray", show_color_bar=False)

    @property
    def node_coordinates(self) -> list[tuple[int, int]]:
        match self._index:
            case 1:
                return np.array([
                    [524, 844],
                    [642, 555],
                ])
            case 2:
                return np.array([
                    [512.5, 734.5],
                    [628, 477.5],
                ])
            case 3:
                return np.array([
                    [594, 747.5],
                    [648.5, 526],
                ])
            case _:
                raise ValueError("Unsupported index")

    @property
    def vectors(self) -> tuple[np.ndarray, np.ndarray]:
        return self.node_coordinates - 512

    @property
    def vector_lengths(self) -> tuple[float, float]:
        return np.linalg.norm(self.node_coordinates[0]-512), np.linalg.norm(self.node_coordinates[1]-512)
    
    @property
    def angle_between_vectors(self) -> float:
        return self.get_angle_between_vectors(self.vectors[0], self.vectors[1])

    @property
    def arrows(self) -> list[gl.Arrow, gl.Text, gl.Arrow, gl.Text]:
        n_coords = self.node_coordinates
        return [
            gl.Arrow((512, 512), (n_coords[0,0], n_coords[0,1]), color="yellow", width=2),
            gl.Text((n_coords[0,0]-512)/2+512+25, (n_coords[0,1]-512)/2+512, "h", color="yellow"),
            gl.Arrow((512, 512), (n_coords[1,0], n_coords[1,1]), color="lime", width=2),
            gl.Text((n_coords[1,0]-512)/2+512, (n_coords[1,1]-512)/2+512-25, "k", color="lime"),
        ]
    
    @staticmethod
    def get_angle_between_vectors(vector_1: np.ndarray, vector_2: np.ndarray) -> float:
        return np.arccos(np.dot(vector_1, vector_2) / (np.linalg.norm(vector_1) * np.linalg.norm(vector_2)))


In [ ]:
# %matplotlib tk
%matplotlib inline

figs = []
for i in range(1,4):
    mg = Micrograph.load(f"design/data/Micrographie_{i}.tif")
    fig = gl.Figure(size=(5, 5), figure_style="dark", title=f"Micrographie {i}", remove_axes=True)
    # fig.add_elements(mg.data_plot)
    fig.add_elements(mg.fft_plot, *mg.arrows)
    figs.append(fig)
    # fig.show()

multifig = gl.MultiFigure.from_row(figs, size=(15, 5), figure_style="dark")
multifig.show()

In [ ]:
for i in range(1,4):
    mg = Micrograph.load(f"design/data/Micrographie_{i}.tif")
    distances = {
        "100" : np.sin(mg.angle_between_vectors)*mg.vector_lengths[0],
        "010" : np.sin(mg.angle_between_vectors)*mg.vector_lengths[1],
        "110" : np.sin(mg.get_angle_between_vectors(mg.vectors[1], mg.vectors[1]-mg.vectors[0]))*mg.vector_lengths[1],
        "1-10" : np.sin(mg.get_angle_between_vectors(mg.vectors[1], mg.vectors[1]+mg.vectors[0]))*mg.vector_lengths[1],
    }

    print(distances)
    print(mg.vectors)
    print(mg.angle_between_vectors)
    print(mg.vector_lengths)
    print()
# print(np.sin(mg.angle_between_vectors)*mg.vector_lengths[0])

\begin{align*}
    d_{100}&=311.43604830202787\textrm{px}
\end{align*}

In [ ]:
def get_fft(index: int) -> gl.Heatmap:
    data = gl.Heatmap(f"design/data/Micrographie_{index}.tif", color_map="gray").image
    fft = np.fft.fft2(data)
    fft_shift = np.fft.fftshift(fft)
    return gl.Heatmap(np.abs(fft_shift), aspect_ratio="equal", vmax=1e5, color_map="gray", show_color_bar=False)

def get_node_coordinates(index: int) -> list[tuple[int, int]]:
    match index:
        case 1:
            return np.array([
                [524, 844],
                [642, 555],
            ])
        case _:
            raise ValueError("Invalid index")

def get_arrows(index: int) -> list[gl.Arrow, gl.Text, gl.Arrow, gl.Text]:
    c = get_node_coordinates(index)
    return [
        gl.Arrow((512, 512), (c[0,0], c[0,1]), color="yellow", width=2),
        gl.Text((c[0,0]-512)/2+512+10, (c[0,1]-512)/2+512, "h", color="yellow"),
        gl.Arrow((512, 512), (c[1,0], c[1,1]), color="lime", width=2),
        gl.Text((c[1,0]-512)/2+512, (c[1,1]-512)/2+512-5, "k", color="lime"),
    ]

def get_vector_lengths(index: int) -> tuple[float, float]:
    c = get_node_coordinates(index)
    return np.linalg.norm(c[0]-512), np.linalg.norm(c[1]-512)


# %matplotlib tk
# %matplotlib inline

# fig = gl.Figure(size=(9, 9))
# fig.add_elements(get_fft(1), *get_arrows(1))
# print(get_vector_lengths(1))
# fig.show()

In [ ]:
# for i in range(1, 4):
#     data = gl.Heatmap(f"design/data/Micrographie_{i}.tif", color_map="gray").image
#     fft = np.fft.fft2(data)
#     fft_shift = np.fft.fftshift(fft)
#     # fft_shift = sp.ndimage.gaussian_filter(np.fft.fftshift(fft), 0.5)

#     hm = gl.Heatmap(np.abs(fft_shift), aspect_ratio="equal", vmax=1e5, color_map="gray")
#     peaks = hm.image > 40000
#     peaks[False] == np.NAN
#     peaks_hm = gl.Heatmap(peaks, color_map="Purples", show_color_bar=False)
#     h_arrow = gl.Arrow((512, 512), (524,844), color="red", width=2)
#     k_arrow = gl.Arrow((512, 512), (642,555), color="green", width=2)

#     # Plot the fft of the image
#     fig = gl.Figure(size=(9, 9))
#     fig.add_elements(hm, h_arrow, k_arrow)
#     fig.show()
#     break
